In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [2]:

# Load the dataset
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
train_dataset, test_dataset, val_dataset = dataset['train'], dataset['test'], dataset['validation']

def preprocess_img(image, label):
    # Ensure image is a 3D tensor
    if image.shape[-1] == 1:
        image = tf.image.grayscale_to_rgb(image)
    elif image.shape[-1] != 3:
        image = tf.image.resize_with_crop_or_pad(image, target_height=256, target_width=256)
    image = tf.image.resize(image, [256, 256])
    # Normalize the image
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Apply the preprocessing function
train_dataset = train_dataset.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and shuffle the training dataset
train_dataset = train_dataset.cache().shuffle(info.splits['train'].num_examples).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


2024-07-19 14:19:11.598815: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-07-19 14:19:11.598839: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-07-19 14:19:11.598843: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-07-19 14:19:11.598861: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-19 14:19:11.598872: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'oxford_flowers102',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [4]:
num_classes = metadata.features['label'].num_classes
class_names = metadata.features['label'].names


In [7]:
for flower in class_names:
    print(flower)
    print("\n")

pink primrose


hard-leaved pocket orchid


canterbury bells


sweet pea


english marigold


tiger lily


moon orchid


bird of paradise


monkshood


globe thistle


snapdragon


colt's foot


king protea


spear thistle


yellow iris


globe-flower


purple coneflower


peruvian lily


balloon flower


giant white arum lily


fire lily


pincushion flower


fritillary


red ginger


grape hyacinth


corn poppy


prince of wales feathers


stemless gentian


artichoke


sweet william


carnation


garden phlox


love in the mist


mexican aster


alpine sea holly


ruby-lipped cattleya


cape flower


great masterwort


siam tulip


lenten rose


barbeton daisy


daffodil


sword lily


poinsettia


bolero deep blue


wallflower


marigold


buttercup


oxeye daisy


common dandelion


petunia


wild pansy


primula


sunflower


pelargonium


bishop of llandaff


gaura


geranium


orange dahlia


pink-yellow dahlia?


cautleya spicata


japanese anemone


black-eyed susan


silverb

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


# Load the dataset
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'oxford_flowers102',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

num_classes = metadata.features['label'].num_classes
class_names = metadata.features['label'].names

# Define preprocessing function
def preprocess_image(image, label):
    image = tf.image.resize_with_pad(image, 224, 224)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Define data augmentation function
def augment(image, label):
    # Flip the image horizontally
    image = tf.image.random_flip_left_right(image)
    
    # Rotate the image
    image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    
    # Adjust brightness
    image = tf.image.random_brightness(image, max_delta=0.2)
    
    # Adjust contrast
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    
    return image, label
# Create train and validation datasets
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32

train_ds = train_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(1000)
train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(AUTOTUNE)

val_ds = val_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE)
val_ds = val_ds.prefetch(AUTOTUNE)


In [7]:
from tensorflow.keras import layers, models

base_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

base_model.trainable = False

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds
)


Epoch 1/30


2024-07-19 14:07:50.385185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 344ms/step - accuracy: 0.0282 - loss: 5.2197 - val_accuracy: 0.1471 - val_loss: 3.7869
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.2383 - loss: 3.3113 - val_accuracy: 0.3627 - val_loss: 2.8373
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - accuracy: 0.4254 - loss: 2.3342 - val_accuracy: 0.4706 - val_loss: 2.2979
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.5538 - loss: 1.7664 - val_accuracy: 0.5294 - val_loss: 1.8719
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.6160 - loss: 1.5348 - val_accuracy: 0.5490 - val_loss: 1.7045
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.6948 - loss: 1.2036 - val_accuracy: 0.5980 - val_loss: 1.6134
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.7609 - loss: 0.9483 - val_accuracy: 0.6176 - val_loss: 1.4610
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.7511 - loss: 0.9245 - val_accuracy: 0.6176 - val

In [9]:
model.save('flower_classifier_model.h5')

In [18]:
from tensorflow.keras.models import load_model


model2 = load_model('flower_classifier_model.h5')



ValueError: Layer "dense" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 2048), dtype=float32, sparse=False, name=keras_tensor_1397>, <KerasTensor shape=(None, 7, 7, 2048), dtype=float32, sparse=False, name=keras_tensor_1398>]

In [10]:
def predict_flower(image):
    img = tf.image.resize(image, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.expand_dims(img, 0)
    prediction = model.predict(img)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction)
    return predicted_class, confidence

In [11]:
import cv2


cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert BGR to RGB (OpenCV uses BGR by default)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get prediction
    flower_name, confidence = predict_flower(rgb_frame)
    
    # Display result on frame
    text = f"{flower_name}: {confidence:.2f}"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Flower Classifier', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━

In [54]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 12.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
